<a href="https://colab.research.google.com/github/DIPANJAN001/Andrew-Ng-Machine-Learning-Notes/blob/master/FinalRLEnvironment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np

In [13]:
class MyEnv3(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=Box(low=np.array([0]),high=np.array([50]),dtype=np.float32) 
        #self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_discharge=15
        self.max_battery_charge=10
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(80,150)
        pv=np.random.randint(110,180)
        battery_state=self.battery_cap
        #net_load=max((load-power),0)
        #net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        #battery_chng=0
        
        if action== 0:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 1:
          grid_export_= max(pv-load-self.max_battery_charge,0) # grid export
          self.battery_cap= max(min(self.battery_cap+min(self.max_battery_charge,pv-load),50),0)
          grid_import_=0
          # only battery charge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
          }
        if action== 2:
            grid_export_=0
            self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)# battery discharge
            grid_import_=max(load-pv-self.max_battery_discharge,0)
            info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
             }
        if action== 3:
          grid_export_=0
          self.battery_cap= min(max(self.battery_cap-min(self.max_battery_discharge,load-pv),0),50)#change in battery
          grid_import_=max(load-pv-self.max_battery_discharge,0)
         #only battery discharge
          info={
              'grid_export':grid_export_,
              'grid_import':grid_import_
                    }     
        
        reward=grid_export_*8-grid_import_*10
        
        if(self.battery_cap<7.0 or self.battery_cap==50):
              done=True
      
        return battery_state,self.battery_cap,load,pv,reward, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap
      

In [14]:
env=MyEnv3()

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [15]:
for e in range(400): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        print(action)
        observation1,observation2,observation3,observation4,reward,done,info = env._step(action)
        print(observation1,observation2,observation3,observation4)
        print(info)
        score+=round(reward)
        
        #print(info['load'],info['power'])
      
    print('episode:{} Profit:{}'.format(e,score))

3
30 50 130 155
{'grid_export': 0, 'grid_import': 0}
episode:0 Profit:0
3
30 50 140 176
{'grid_export': 0, 'grid_import': 0}
episode:1 Profit:0
2
30 50 125 154
{'grid_export': 0, 'grid_import': 0}
episode:2 Profit:0
2
30 50 148 168
{'grid_export': 0, 'grid_import': 0}
episode:3 Profit:0
3
30 47 95 112
{'grid_export': 0, 'grid_import': 0}
0
47 50 118 138
{'grid_export': 10, 'grid_import': 0}
episode:4 Profit:80
3
30 47 134 151
{'grid_export': 0, 'grid_import': 0}
0
47 50 121 173
{'grid_export': 42, 'grid_import': 0}
episode:5 Profit:336
1
30 36 113 119
{'grid_export': 0, 'grid_import': 0}
3
36 39 128 131
{'grid_export': 0, 'grid_import': 0}
0
39 49 109 175
{'grid_export': 56, 'grid_import': 0}
1
49 50 124 174
{'grid_export': 40, 'grid_import': 0}
episode:6 Profit:768
3
30 50 80 117
{'grid_export': 0, 'grid_import': 0}
episode:7 Profit:0
1
30 40 138 163
{'grid_export': 15, 'grid_import': 0}
3
40 50 117 146
{'grid_export': 0, 'grid_import': 0}
episode:8 Profit:120
1
30 24 127 121
{'grid_e

### Desigining the agent